In [6]:
import os 
import yaml
def read_config(path):
    """
    Reads API key from a configuration file.

    This function opens a configuration file named "apikeys.yml", reads the API key for OpenAI

    Returns:
    api_key (str): The API key for the Amadeus Flights API.
    """
    
    # Get the directory of the current script
    script_dir = path

    # Construct the full path to the configuration file
    file_path = os.path.join(script_dir, "apikeys.yml")

    with open(file_path, 'r') as stream:
        configs = yaml.safe_load(stream)
        API_KEY = configs['openai']['api_key']
            
    return API_KEY
path = r"C:\Users\johna\OneDrive\Documents\api_keys"  # Change to the location of your apikeys.yml
API_KEY = read_config(path)

In [4]:
from haystack.nodes import PreProcessor
from haystack.utils import convert_files_to_docs
from haystack.document_stores import FAISSDocumentStore
from sqlalchemy import create_engine

# pre-process docs 
def preprocess_docs(doc_dir):
    all_docs = convert_files_to_docs(dir_path=doc_dir)
    preprocessor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=False,
        split_by="word",
        split_respect_sentence_boundary=True,
        split_overlap=30, 
        split_length=100
    )
    docs = preprocessor.process(all_docs)
    print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")
    return docs


# create FAISS
def vector_stores(docs):
    engine = create_engine('sqlite:///C:/Users/johna/anaconda3/envs/longfunctioncall_env/long_functioncall/database/database.db')  # change to your local directory
    try:
        # Attempt to drop the table
        engine.execute("DROP TABLE document")
    except Exception as e:
        # Catch any exceptions, likely due to the table not existing
        print(f"Exception occurred while trying to drop the table: {e}")
    
    document_store = FAISSDocumentStore(sql_url='sqlite:///C:/Users/johna/anaconda3/envs/longfunctioncall_env/long_functioncall/database/database.db', faiss_index_factory_str="Flat", embedding_dim=768) # change to your local directory
    document_store.write_documents(docs)
    
    return document_store

In [5]:
doc_dir = r"C:\Users\johna\anaconda3\envs\longfunctioncall_env\long_functioncall\knowledge_base"
docs = preprocess_docs(doc_dir)
document_store = vector_stores(docs)

Preprocessing: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.61docs/s]


n_files_input: 1
n_docs_output: 1176
Exception occurred while trying to drop the table: (sqlite3.OperationalError) no such table: document
[SQL: DROP TABLE document]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Writing Documents: 10000it [00:02, 3442.53it/s]                                                                        


In [26]:
import openai
from haystack.nodes.base import BaseComponent
from typing import List

class OpenAIFunctionCall(BaseComponent):
    outgoing_edges = 1

    def run(self, documents: List[str]):

        # Print the type of 'documents' and the first few elements
        print("Documents type:", type(documents))
        print("First few documents:", documents[:2])
    
        # Try to extract the content and print the first few content strings
        try:
            document_content_list = [doc.content for doc in documents]
            print("First few content strings:", document_content_list[:2])
            document_content = " ".join(document_content_list)
        except Exception as e:
            print("Error extracting content:", e)
            return

        # concatenate docs
        # document_content = " ".join([doc.content for doc in documents])

        functions = [
            {
                "name": "write_to_df",
                "description": "write the fund details to a dataframe",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "prr": {
                            "type": "string",
                            "description": "The FCA product reference number which will be six or seven digits"
                        },
                        "investment_objective": {
                            "type": "string",
                            "description": "You should return the investment objective of the fund. This is likely to be something like this: The Fund aims to grow your investment over t – t + delta t years"
                        },
                        "investment_policy": {
                            "type": "string",
                            "description": "Return a summary of the fund's investment policy, no more than two sentences."
                        },
                        "investment_strategy": {
                            "type": "string",
                            "description": "Return a summary of the fund's investment strategy, no more than two sentences."
                        },
                        "ESG": {
                            "type": "string",
                            "description": "Return either True, or False. True if the fund is an ESG fund, False otherwise."
                        },
                    },
                },
                "required": ["prr", 
                             "investment_objective", 
                             "investment_policy",
                             "investment_strategy",
                             "ESG"]
            }
        ]
        
        openai.api_key = API_KEY
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[{"role": "system", "content": document_content}],
            functions=functions,
            function_call="auto",  # auto is default, but we'll be explicit
        )

        print("OPENAI RESPONSE!!",response)

        function_call_args = json.loads(response["choices"][0]["message"]["function_call"]["arguments"])


        # response_message = response["choices"][0]["message"]["content"]

        return response, "output_1"

    def run_batch(self, documents: List[str]):
        # You can either process multiple documents in a batch here or simply loop over the run method
        results = []
        for document_content in document_content:
            result, _ = self.run(document_content)
            results.append(result)
        return results, "output_1"


In [2]:
# # define custom note OpenAIFunctionCall for our pipeline

# import openai
# from haystack.nodes.base import BaseComponent

# class OpenAIFunctionCall(BaseComponent):
#     outgoing_edges=1

#     def run(self, document: str):
#         functions = [
#         {
#             "name": "write_to_df",
#             "description": "write the fund details to a dataframe",
#             "parameters": {
#                 "type": "object",
#                 "properties": {
#                     "prr": {
#                         "type": "string",
#                         "description": "The FCA product reference number which will be six or seven digits"
#                     },
#                     "investment_objective": {
#                         "type": "string",
#                         "description": """You should return a the investment objective of the fund.
#                         This is likely to be something like this: The Fund aims to grow your investment over t – t + delta t years
#                         """
#                     },
#                     "investment_policy": {
#                         "type": "string",
#                         "description": """ Return a summary of the fund's investment policy, no more than a two sentences.
#                         """
#                     },
#                     "investment_strategy": {
#                         "type": "string",
#                         "description": """ Return a summary of the fund's investment strategy, no more than a two sentences.
#                         """
#                     },
#                     "ESG": {
#                         "type": "string",
#                         "description": """ Return either True, or False. True if the fund is an ESG fund, False otherwise.
#                         """
#                     },
                    
#                  },
#                 },
#                 "required": ["prr", 
#                              "investment_objective", 
#                              "investment_policy",
#                              "investment_strategy",
#                              "ESG"]
#             }
#         ]
        
#         openai.api_key = API_KEY
#         response = openai.ChatCompletion.create(
#         model="gpt-3.5-turbo-0613",
#         messages=[{"role": "system", "content": document}],
#         functions=functions,
#         function_call="auto",  # auto is default, but we'll be explicit
#         )
#         response_message = response["choices"][0]["message"]

#         return response_message
        

C:\Users\johna\anaconda3\envs\longfunctioncall_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# create our pipeline
from haystack import Pipeline
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/all-mpnet-base-v2"
)
document_store.update_embeddings(retriever)

In [27]:
p = Pipeline()
p.add_node(component=retriever, name="retriever", inputs=["Query"])
p.add_node(component=OpenAIFunctionCall(), name="OpenAIFunctionCall", inputs=["retriever"])
res = p.run(query="The FCA product reference number which will be six or seven digits")
res

Batches: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.87it/s]


Documents type: <class 'list'>
First few documents: [<Document: {'content': 'The FCA’s product reference number\nfor the Company is 186580. The product reference number of each of the Funds is set out\nbelow.\nShareholders of the Company are not liable for the debts of the Company.\nThe head office of the Company is at 25 Cabot Square, Canary Wharf, London E14 4QA.\nThe head office is the address of the place in the United Kingdom for service on the\nCompany of notices or other documents required or authorised to be served on it.\nBase Currency\nThe base currency of the Company is pounds sterling. ', 'content_type': 'text', 'score': 0.5014380682094575, 'meta': {'name': 'prospectus_msfundsuk_en.pdf', '_split_id': 34, '_split_overlap': [{'doc_id': '6a95ed9bf8517f35e96d303a05c2b3f5', 'range': [0, 202]}, {'doc_id': 'b3b6342c6a91414b478e3090b0d8dde0', 'range': [287, 519]}], 'vector_id': '58'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '1e2ceebb46ac94c977054d2141d73406'}>, <Docum

TypeError: Object of type Document is not JSON serializable

In [ ]:
# from haystack.nodes import EmbeddingRetriever, FARMReader
# from haystack.pipelines import ExtractiveQAPipeline
# from haystack.pipelines import Pipeline


# # retreiver relevant docs
# def make_document_qa_pipeline(document_store):
#     retriever = EmbeddingRetriever(
#         document_store=document_store,
#         embedding_model="sentence-transformers/all-mpnet-base-v2"
#     )
#     document_store.update_embeddings(retriever)
    
#     # read relevant docs
#     reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")
    
#     document_qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)
#     return document_qa